<a href="https://colab.research.google.com/github/aaronstone1310/Verification-of-Stabilizer-Evolution/blob/main/Stabilizer_evolution_MBQC_Shor_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class ClusterState:

    nodes = None
    edges = None
    adjecency_matrix = None
    num_nodes = None
    num_qubits = None
    H_X = None
    H_Z = None

    def __init__(self, nodes, edges):

        self.nodes = nodes
        self.edges = edges
        self.num_nodes = len(nodes)
        self.adjecency_matrix = self._set_adjecency_matrix(len(nodes), edges)
        self.H_X, self.H_Z = self.parity_check_matrix()

    def _set_adjecency_matrix(self, dimension, edges):

        adjecency_matrix = np.zeros((dimension, dimension))
        for e in edges:
            i, j = e
            adjecency_matrix[int(i)-1][int(j)-1] = 1
            adjecency_matrix[int(j)-1][int(i)-1] = 1

        return adjecency_matrix

    def parity_check_matrix(self):

        H_X = np.zeros((self.num_nodes, self.num_nodes))
        H_Z = np.zeros((self.num_nodes, self.num_nodes))

        H_Z = self.adjecency_matrix

        for i in range(len(H_X)):
            H_X[i][i] = 1

        return H_X, H_Z

    @staticmethod
    def stabilizer_generators(H_X, H_Z):

        dim = H_X.shape[-1]
        generators = []

        for i in range(len(H_X)):
            stabilizer = ["I"]*dim
            for j in range(dim):
                if H_Z[i][j] == 1:
                    stabilizer[j] = "Z"
                if H_X[i][j] == 1:
                    stabilizer[j] = "X"
            stabilizer = "".join(stabilizer)
            generators.append(stabilizer)

        return generators

    @staticmethod
    def offset(n, lst):

        if len(lst) == 0:
            return 0
        else:
            off = list(filter(lambda x: n > x, lst))
            count = len(off)
            return count

    @staticmethod
    def XOR(arr1, arr2):
        arr = arr1 != arr2
        arr = arr.astype(int)
        return arr

    def measure(self, measurement):


        updated_h_x = np.copy(self.H_X)
        updated_h_z = np.copy(self.H_Z)

        qubits_measured = []

        for q, m in measurement.items():
            p = int(q)-1
            anti_commute = []

            p1 = p - ClusterState.offset(p,qubits_measured)

            for i in range(len(updated_h_z)):
                if updated_h_z[i][p1] == 1:
                    anti_commute.append(i)

            i1 = anti_commute[0]

            for i in range(1,len(anti_commute)):
                i2 = anti_commute[i]
                x = ClusterState.XOR(updated_h_x[i1], updated_h_x[i2])
                z = ClusterState.XOR(updated_h_z[i1], updated_h_z[i2])
                for i in range(len(x)):
                    updated_h_x[i2][i] = x[i]
                    updated_h_z[i2][i] = z[i]

            qubits_measured.append(p1)

            h_x = np.delete(updated_h_x,i1,0)
            h_z = np.delete(updated_h_z,i1,0)
            h_x = np.delete(h_x,p1,1)
            h_z = np.delete(h_z,p1,1)
            updated_h_x = h_x
            updated_h_z = h_z

        return updated_h_x, updated_h_z

In [ ]:
nodes = [str(i) for i in range(1,17)]

edges = [('1','2'),('1','3'),('1','4'),('2','5'),
     ('2','7'),('3','9'),('3','11'),('4','13'),
     ('4','15'),('5','6'),('7','8'),('9','10'),
     ('11','12'),('13','14'),('15','16')]

cs = ClusterState(nodes,edges)
sg = cs.stabilizer_generators(cs.H_X, cs.H_Z)
sg

['XZZZIIIIIIIIIIII',
 'ZXIIZIZIIIIIIIII',
 'ZIXIIIIIZIZIIIII',
 'ZIIXIIIIIIIIZIZI',
 'IZIIXZIIIIIIIIII',
 'IIIIZXIIIIIIIIII',
 'IZIIIIXZIIIIIIII',
 'IIIIIIZXIIIIIIII',
 'IIZIIIIIXZIIIIII',
 'IIIIIIIIZXIIIIII',
 'IIZIIIIIIIXZIIII',
 'IIIIIIIIIIZXIIII',
 'IIIZIIIIIIIIXZII',
 'IIIIIIIIIIIIZXII',
 'IIIZIIIIIIIIIIXZ',
 'IIIIIIIIIIIIIIZX']

In [ ]:
measurement = {"1":"X", "5":"X", "7":"X", "9":"X", "11":"X", "13":"X", "15":"X"}
final_h_x, final_h_z = cs.measure(measurement)
ls = [final_h_x, final_h_z]
final_h_x, final_h_z

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 0., 1., 1.]]),
 array([[1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [ ]:
final_state = cs.stabilizer_generators(final_h_x, final_h_z)
final_state

['ZZZIIIIII',
 'ZIIZIIIII',
 'ZIIIZIIII',
 'IZIIIZIII',
 'IZIIIIZII',
 'XXIXXXXII',
 'IIZIIIIZI',
 'IIZIIIIIZ',
 'XIXXXIIXX']